In [1]:
import networkx as nx
import matplotlib.pyplot as plt

# 1. Load up the directed multigraph

In [2]:
def load_directed_multigraph_from_txt_file(txt):
    """ Creates and loads a graph from  """
    G = nx.read_edgelist(txt, create_using=nx.MultiDiGraph(), nodetype = str)
    print(nx.info(G))
    

    
load_directed_multigraph_from_txt_file("./email-EuAll.txt")



Name: 
Type: MultiDiGraph
Number of nodes: 265214
Number of edges: 420045
Average in degree:   1.5838
Average out degree:   1.5838


# 2. How many unique sources did send an email? And how many emails were sent?

In [3]:
def retrive_number_of_unique_sources_that_sent_an_email():
    pass
def retrive_number_of_emails_sent_by_unique_sources():
    pass